In [1]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

True
1
NVIDIA GeForce RTX 2050


In [2]:
%pip install -q -U einops datasets matplotlib tqdm

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import math #useful for sine cos etc
from inspect import isfunction #check if stuff is func
from functools import partial #allows you to define functions using other functions

%matplotlib inline
import matplotlib.pyplot as plt #Standard
from tqdm.auto import tqdm #pretty progress bars
from einops import rearrange, reduce #tensor operations
from einops.layers.torch import Rearrange #layer based rather than tensor

import torch
from torch import nn, einsum #einsum is a kind of notation
import torch.nn.functional as F #useful funcs like relu sigmoid, mse cs, ppoling and dropout 



c:\Akshay\Projects\diffusion-implementation\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


right now we have gotten all the imports done
The very first step in the ddpm is to define a variance schedule
This acts as a hyperparameter of β1 β2 ... βT where T=1000 is a hyperparameter
T= 1000 in the original DDPM paper.

Recall that the sequence of "noisening" the input images is following the equation 

![gaussian noise equation](images/eq1.png)

The above equation refers to the forward  process, in this process we convert an image to a slightly noisier representation of itself
The variable xt refers to the input image x0 at timestamp t in the variance schedule (β1....βt....βT).
Therefore the t'th image relies on the t-1'th image. It is sampled from the normal distribution, where the mean is
root(1-β t) (often abbreviated as root(alpha t))
and covariance matrix is β t * I (I is identity matrix)

Now you might be asking:
> Why do we use a matrix where a variance should be there?

This is because the xt might be a multivariate distribution. What does this mean simply? There may be multiple features and dimensions to xt. Multiplying by I essentially sets the "variance of that particular dimension/feature" to β t.


Note that the βt aren't constant at each time step t (hence the subscript) --- in fact one defines a so-called "variance schedule", which can be linear, quadratic, cosine, etc. as we will see further (a bit like a learning rate schedule).
